# MICCAI 2022

In [ ]:
import os
import glob
import random
import torch
import numpy as np
import matplotlib.pyplot as plt
import skimage.io as io

# Imports for image preprocessing
from skimage import io, exposure
from skimage.transform import resize
from skimage.draw import disk

# For calculating SSIM
from ssim import ssim

In [ ]:
# Allow loading of truncated images
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:

def load_stack(stack_dir_path):
    """ Load a stack from the containing directory path. """
    img_paths = glob.glob(os.path.join(stack_dir_path, '*.jpg'))
    # Create list of images tagged with their focal depths
    imgs = [
        (
            int(os.path.splitext(os.path.basename(img_path))[0][1:]),
            io.imread(img_path, as_gray=True)
        )
        for img_path in img_paths
    ]
    # Sort by focal depth
    imgs = sorted(imgs, key=lambda x: x[0])
    # Remove the focal depth info
    imgs = [img for _, img in imgs]
    return imgs


In [ ]:
def normalise_stack(imgs, plane_size):
    """ Apply normalisation across the whole stack by combining planes
        into single image and applying normalisation to that. """
    # Combine the images
    combined_img = np.zeros(
        (plane_size[0] * len(imgs), plane_size[1]))
    for i in range(len(imgs)):
        combined_img[i * plane_size[0]:(i + 1) * plane_size[0], 0:plane_size[1]] = imgs[i]
    # Normalise combined image
    combined_img = exposure.equalize_adapthist(
        combined_img, clip_limit=0.01)
    # Unpack the combined image into planes
    return [combined_img[i * plane_size[0]:(i + 1) * plane_size[0], 0:plane_size[1]]
            for i in range(len(imgs))]

In [ ]:

def preprocess_stack(imgs, plane_size=(400, 400), crop_proportion=0.0625):
    """ Preprocess a stack. """
    # Generate mask for hiding the well outline
    rr, cc = disk((plane_size[0]/2, plane_size[1]/2), plane_size[0]/2)
    circle_mask = np.zeros(plane_size)
    circle_mask[rr, cc] = 1
    # Center-crop, resize and apply mask to all images
    h, w = imgs[0].shape
    h_rm, w_rm = int(h * crop_proportion), int(w * crop_proportion)
    imgs = [img[h_rm:-h_rm, w_rm:-w_rm] for img in imgs]
    imgs = [resize(img, plane_size) for img in imgs]
    imgs = [img * circle_mask for img in imgs]
    # Normalise stack
    imgs = normalise_stack(imgs, plane_size)
    return imgs


In [ ]:
def predict_with_model(model, first, second):
    image = np.zeros((400, 400, 2))
    image[:, :, 0] = first
    image[:, :, 1] = second
    image = torch.from_numpy(image).permute(2, 0, 1).unsqueeze(0).float()
    image = image.cuda() if torch.cuda.is_available() else image
    pred = model.forward(image)
    pred = pred.squeeze().detach().cpu()
    return pred


In [ ]:
rr, cc = disk((200, 200), 200)
circle_mask = np.zeros((400, 400))
circle_mask[rr, cc] = 1

In [ ]:

def generate_stack_from_path(decreasing_model, increasing_model, middle_model, stack_dir, metric_fn=lambda *x: 0):
    # Load the stack
    stack = preprocess_stack(load_stack(stack_dir))
    # Set output directory
    output_path = os.path.basename(stack_dir)
    focals = ['F-75', 'F-60', 'F-45', 'F-30', 'F-15',
              'F0', 'F15', 'F30', 'F45', 'F60', 'F75']
    scores = []
    # Randomly switch between in-between generation and extrapolation
    if random.random() < 0.5:
        # Pick random plane to remove (except for the ends)
        idxs_to_remove = random.choices(range(1,len(focals) - 1), k=2)
        focals_to_remove = [focals[idx] for idx in idxs_to_remove]
        # Generate the missing ones
        first_missing = predict_with_model(
            middle_model, stack[idxs_to_remove[0] + 1], stack[idxs_to_remove[0] - 1]
        ) * circle_mask
        score = metric_fn(first_missing.unsqueeze(0).unsqueeze(0).float(), torch.tensor(stack[idxs_to_remove[0]]).unsqueeze(0).unsqueeze(0).float()) 
        first_missing[1:20, 1:20] = 1
        second_missing = predict_with_model(
            middle_model, stack[idxs_to_remove[1] + 1], stack[idxs_to_remove[1] - 1]
        ) * circle_mask
        score += metric_fn(second_missing.unsqueeze(0).unsqueeze(0).float(), torch.tensor(stack[idxs_to_remove[1]]).unsqueeze(0).unsqueeze(0).float()) 
        score /= 2
        second_missing[1:20, 1:20] = 1
        # Generate the missing planes
        io.imsave(os.path.join(output_path, f'{focals_to_remove[0]}.jpg'), first_missing.numpy())
        io.imsave(os.path.join(output_path, f'{focals_to_remove[1]}.jpg'), second_missing.numpy())
    else:
        # Generate missing planes and mark them as fake with a lil square in the top corner
        focals = ['F-75', 'F-60', 'F-45', 'F-30', 'F-15',
              'F0', 'F15', 'F30', 'F45', 'F60', 'F75']
        plane_plus_60 = predict_with_model(
            increasing_model, stack[8], stack[7]
        ) * circle_mask
        plane_plus_75 = predict_with_model(
            increasing_model, plane_plus_60, stack[8]
        ) * circle_mask
        scores.append(metric_fn(plane_plus_60.unsqueeze(0).unsqueeze(0).float(), torch.tensor(stack[9]).unsqueeze(0).unsqueeze(0).float()))
        scores.append(metric_fn(plane_plus_75.unsqueeze(0).unsqueeze(0).float(), torch.tensor(stack[10]).unsqueeze(0).unsqueeze(0).float()))
        plane_plus_60[1:20, 1:20] = 1
        plane_plus_75[1:20, 1:20] = 1
        plane_minus_60 = predict_with_model(
            decreasing_model, stack[2], stack[3]
        )
        plane_minus_75 = predict_with_model(
            decreasing_model, plane_minus_60, stack[2]
        )
        scores.append(metric_fn(plane_plus_60.unsqueeze(0).unsqueeze(0).float(), torch.tensor(stack[9]).unsqueeze(0).unsqueeze(0).float()))
        scores.append(metric_fn(plane_plus_75.unsqueeze(0).unsqueeze(0).float(), torch.tensor(stack[10]).unsqueeze(0).unsqueeze(0).float()))
        plane_minus_60[1:20, 1:20] = 1
        plane_minus_75[1:20, 1:20] = 1
        # Save the new stack
        io.imsave(os.path.join(output_path, 'F60.jpg'), plane_plus_60.numpy())
        io.imsave(os.path.join(output_path, 'F75.jpg'), plane_plus_75.numpy())
        io.imsave(os.path.join(output_path, 'F-60.jpg'),
                  plane_minus_60.numpy())
        io.imsave(os.path.join(output_path, 'F-75.jpg'),
                  plane_minus_75.numpy())

    # Save the new stack
    for i, img in enumerate(stack):
        io.imsave(os.path.join(output_path, f'{focals[i]}_gt.jpg'), img)

    return scores


In [ ]:
# Load models
decreasing_model = torch.load('PLACEHOLDER')
decreasing_model = decreasing_model.cuda(
) if torch.cuda.is_available() else decreasing_model
decreasing_model.eval()
increasing_model = torch.load('PLACEHOLDER')
increasing_model = increasing_model.cuda(
) if torch.cuda.is_available() else increasing_model
increasing_model.eval()
middle_model = torch.load('PLACEHOLDER')
middle_model = middle_model.cuda(
) if torch.cuda.is_available() else middle_model
middle_model.eval()

In [ ]:
from tqdm import tqdm
# Find all the stacks of interest
stack_dirs = glob.glob('PLACEHOLDER')[20:220]
model_paths = [
    # TODO: ADD MODELS
]
for model_path in model_paths:
    scores = []
    model = torch.load(model_path)
    model = model.cuda() if torch.cuda.is_available() else model
    model.eval()
    # Loop through and generate
    for i, stack_dir in tqdm(enumerate(stack_dirs)):
        # Load the stack
        stack = preprocess_stack(load_stack(stack_dir))
        # Generate them all
        for idx in range(2, len(stack)):
            plane = predict_with_model(
                model, stack[idx - 1], stack[idx - 2]
            ) * circle_mask
            actual_plane = stack[idx]
            scores.append(ssim(plane.unsqueeze(0).unsqueeze(0).cuda(), torch.tensor(stack[idx]).unsqueeze(0).unsqueeze(0).cuda()).item())
    print(np.mean(scores), np.std(scores))

In [ ]:
from tqdm import tqdm
# Find all the stacks of interest
stack_dirs = glob.glob('PLACEHOLDER')[20:220]
model_paths = [
    # TODO: ADD MODELS
]
for model_path in model_paths:
    scores = []
    model = torch.load(model_path)
    model = model.cuda() if torch.cuda.is_available() else model
    model.eval()
    # Loop through and generate
    for i, stack_dir in tqdm(enumerate(stack_dirs)):
        # Load the stack
        stack = preprocess_stack(load_stack(stack_dir))
        # Generate them all
        for idx in range(2, len(stack)):
            plane = predict_with_model(
                model, stack[idx - 1], stack[idx - 2]
            ) * circle_mask
            actual_plane = stack[idx]
            mse = torch.mean((plane - torch.tensor(stack[idx]))**2).item()
            scores.append(10 * np.log10(1.0 / mse))
    print(np.mean(scores), np.std(scores))

#### FID

In [ ]:
from tqdm import tqdm
# Find all the stacks of interest
stack_dirs = glob.glob('PLACEHOLDER')[20:4020]
scores = []
model_paths = [
    # TODO: ADD MODELS
]
k = 0
for j, model_path in enumerate(model_paths):
    os.mkdir(str(j))
    os.mkdir(os.path.join(str(j), 'real'))
    os.mkdir(os.path.join(str(j), 'fake'))
    model = torch.load(model_path)
    model = model.cuda() if torch.cuda.is_available() else model
    model.eval()
    # Loop through and generate
    for i, stack_dir in enumerate(stack_dirs):
        k += 1
        # Load the stack
        stack = preprocess_stack(load_stack(stack_dir))
        # Generate them all
        for idx in range(2, len(stack)):
            plane = predict_with_model(
                model, stack[idx - 1], stack[idx - 2]
            ) * circle_mask
            actual_plane = stack[idx]
            io.imsave(os.path.join(str(j), 'real', f'{k}.jpg'), plane.numpy())
            io.imsave(os.path.join(str(j), 'fake', f'{k}.jpg'), stack[idx])


# Qualitative Experiments

### PLACEHOLDER (7 -> 11)

In [ ]:
# Find all the stacks of interest
stack_dirs = glob.glob('PLACEHOLDER')
# Filter out the 11 FP ones
stack_dirs = [d for d in stack_dirs if len(glob.glob(f'{d}/*')) == 7]
os.mkdir('PLACEHOLDER')
# Loop through and generate
for i, stack_dir in enumerate(random.sample(stack_dirs, 10)):
    # Load the stack
    stack = preprocess_stack(load_stack(stack_dir))
    print(len(stack))
    plane_plus_60 = predict_with_model(
            increasing_model, stack[6], stack[5]
        ) * circle_mask
    plane_plus_75 = predict_with_model(
            increasing_model, plane_plus_60, stack[6]
        ) * circle_mask
    plane_plus_60[1:20, 1:20] = 1
    plane_plus_75[1:20, 1:20] = 1
    plane_minus_60 = predict_with_model(
        decreasing_model, stack[0], stack[1]
    )
    plane_minus_75 = predict_with_model(
        decreasing_model, plane_minus_60, stack[0]
    )
    plane_minus_60[1:20, 1:20] = 1
    plane_minus_75[1:20, 1:20] = 1
    # Save the new stack
    output_path = os.path.join('PLACEHOLDER', str(i))
    os.mkdir(output_path)
    io.imsave(os.path.join(output_path, 'F60.jpg'), plane_plus_60.numpy())
    io.imsave(os.path.join(output_path, 'F75.jpg'), plane_plus_75.numpy())
    io.imsave(os.path.join(output_path, 'F-60.jpg'),
                plane_minus_60.numpy())
    io.imsave(os.path.join(output_path, 'F-75.jpg'),
                plane_minus_75.numpy())
    # Add existing planes
    focals = ['F-45', 'F-30', 'F-15',
        'F0', 'F15', 'F30', 'F45']
    for j, img in enumerate(stack):
        io.imsave(os.path.join(output_path, f'{focals[j]}.jpg'), img)

### PLACEHOLDER - (shift)

In [ ]:
# Find all the stacks of interest
stack_dirs = glob.glob('PLACEHOLDER')
os.mkdir('PLACEHOLDER')
# Loop through and generate
for i, stack_dir in enumerate(random.sample(stack_dirs, 10)):
    # Load the stack
    stack = preprocess_stack(load_stack(stack_dir))
    plane_plus_90 = predict_with_model(
            increasing_model, stack[10], stack[9]
        ) * circle_mask
    plane_plus_105 = predict_with_model(
            increasing_model, plane_plus_90, stack[10]
        ) * circle_mask
    plane_plus_90[1:20, 1:20] = 1
    plane_plus_105[1:20, 1:20] = 1
    # Save the new stack
    output_path = os.path.join('PLACEHOLDER', str(i))
    os.mkdir(output_path)
    io.imsave(os.path.join(output_path, 'F90.jpg'), plane_plus_90.numpy())
    io.imsave(os.path.join(output_path, 'F105.jpg'), plane_plus_105.numpy())
    # Add existing planes
    focals = ['F-45', 'F-30', 'F-15',
        'F0', 'F15', 'F30', 'F45', 'F60', 'F75']
    for j, img in enumerate(stack[2:]):
        io.imsave(os.path.join(output_path, f'{focals[j]}.jpg'), img)

### PLACEHOLDER (Random Removal of N planes)

In [ ]:
# Find all the stacks of interest
stack_dirs = glob.glob('PLACEHOLDER')[20:]
os.mkdir('PLACEHOLDER')
os.mkdir('PLACEHOLDER')
# Loop through and generate
for i, stack_dir in enumerate(random.sample(stack_dirs, 10)):
    focals = ['F-75', 'F-60', 'F-45', 'F-30', 'F-15',
              'F0', 'F15', 'F30', 'F45', 'F60', 'F75']
    # Load the stack
    stack = preprocess_stack(load_stack(stack_dir))
    print(len(stack))
    # Pick random planes to remove
    idxs_to_remove = random.sample(range(len(focals)), 4)
    focals_to_remove = [focals[idx] for idx in idxs_to_remove]
    for focal in focals_to_remove:
        try:
            focals.remove(focal)
        except Exception:
            print('ERROR REMOVING FOCAL')
            continue
    # Generate the missing ones
    output_path = os.path.join('PLACEHOLDER-random', str(i))
    output_path_orig = os.path.join('PLACEHOLDER-random-orig', str(i))
    os.mkdir(output_path)
    os.mkdir(output_path_orig)
    for j, idx in enumerate(idxs_to_remove):
        if idx == 0:
            plane = predict_with_model(
                decreasing_model, stack[idx + 1], stack[idx + 2]
            ) * circle_mask
        elif idx == len(stack) - 1:
            plane = predict_with_model(
                increasing_model, stack[idx - 1], stack[idx - 2]
            ) * circle_mask
        else:
            plane = predict_with_model(
                middle_model, stack[idx + 1], stack[idx - 1]
            ) * circle_mask
        plane[1:20, 1:20] = 1
        io.imsave(os.path.join(output_path, f'{focals_to_remove[j]}.jpg'), plane.numpy())
        io.imsave(os.path.join(output_path_orig, f'{focals_to_remove[j]}.jpg'), stack[idx])
    for j, img in enumerate(stack):
        if j not in idxs_to_remove:
            io.imsave(os.path.join(output_path, f'{focals[j - len([x for x in idxs_to_remove if x < j])]}.jpg'), img)

### PLACEHOLDER (3 -> 11)

In [ ]:
# Find all the stacks of interest
stack_dirs = glob.glob('PLACEHOLDER')
os.mkdir('PLACEHOLDER')
# Loop through and generate
for i, stack_dir in enumerate(random.sample(stack_dirs, 10)):
    # Load the stack
    stack = preprocess_stack(load_stack(stack_dir))
    print(len(stack))
    plane_plus_15 = predict_with_model(
        middle_model, stack[2], stack[1]
    ) * circle_mask
    plane_minus_15 = predict_with_model(
        middle_model, stack[1], stack[0]
    ) * circle_mask
    plane_plus_45 = predict_with_model(
        increasing_model, stack[2], plane_plus_15
    ) * circle_mask
    plane_plus_60 = predict_with_model(
        increasing_model, plane_plus_45, stack[2]
    ) * circle_mask
    plane_plus_75 = predict_with_model(
        increasing_model, plane_plus_60, plane_plus_45,
    ) * circle_mask
    plane_minus_45 = predict_with_model(
        decreasing_model, stack[0], plane_minus_15
    ) * circle_mask
    plane_minus_60 = predict_with_model(
        decreasing_model, plane_minus_45, stack[0]
    ) * circle_mask
    plane_minus_75 = predict_with_model(
        decreasing_model, plane_minus_60, plane_minus_45
    ) * circle_mask
    
    plane_plus_15[1:20, 1:20] = 1
    plane_plus_45[1:20, 1:20] = 1
    plane_plus_60[1:20, 1:20] = 1
    plane_plus_75[1:20, 1:20] = 1
    plane_minus_15[1:20, 1:20] = 1
    plane_minus_45[1:20, 1:20] = 1
    plane_minus_60[1:20, 1:20] = 1
    plane_minus_75[1:20, 1:20] = 1
    # Save the new stack
    output_path = os.path.join('PLACEHOLDER-insane', str(i))
    os.mkdir(output_path)
    io.imsave(os.path.join(output_path, 'F15.jpg'), plane_plus_15.numpy())
    io.imsave(os.path.join(output_path, 'F45.jpg'), plane_plus_45.numpy())
    io.imsave(os.path.join(output_path, 'F60.jpg'), plane_plus_60.numpy())
    io.imsave(os.path.join(output_path, 'F75.jpg'), plane_plus_75.numpy())
    io.imsave(os.path.join(output_path, 'F-15.jpg'),
                plane_minus_15.numpy())
    io.imsave(os.path.join(output_path, 'F-45.jpg'),
                plane_minus_45.numpy())
    io.imsave(os.path.join(output_path, 'F-60.jpg'),
                plane_minus_60.numpy())
    io.imsave(os.path.join(output_path, 'F-75.jpg'),
                plane_minus_75.numpy())
    # Add existing planes
    focals = ['F-30', 'F0', 'F30']
    for j, img in enumerate(stack):
        io.imsave(os.path.join(output_path, f'{focals[j]}.jpg'), img)

### PLACEHOLDER (Fully generated for blast grading)

In [ ]:
# Find all the stacks of interest
stack_dirs = glob.glob('PLACEHOLDER')[20:]
os.mkdir('PLACEHOLDER')
# Loop through and generate
for i, stack_dir in enumerate(random.sample(stack_dirs, 50)):
    focals = ['F-75', 'F-60', 'F-45', 'F-30', 'F-15',
              'F0', 'F15', 'F30', 'F45', 'F60', 'F75']
    # Load the stack
    stack = preprocess_stack(load_stack(stack_dir))
    print(len(stack))
    # Generate them all
    output_path = os.path.join('PLACEHOLDER', f'{i}_fake')
    os.mkdir(output_path)
    for idx in range(len(stack)):
        if idx == 0:
            plane = predict_with_model(
                decreasing_model, stack[idx + 1], stack[idx + 2]
            ) * circle_mask
        elif idx == len(stack) - 1:
            plane = predict_with_model(
                increasing_model, stack[idx - 1], stack[idx - 2]
            ) * circle_mask
        else:
            plane = predict_with_model(
                middle_model, stack[idx + 1], stack[idx - 1]
            ) * circle_mask
        io.imsave(os.path.join(output_path, f'{focals[idx]}.jpg'), plane.numpy())
    # Generate the GT
    output_path = os.path.join('PLACEHOLDER', f'{i}')
    os.mkdir(output_path)
    for idx in range(len(stack)):
        io.imsave(os.path.join(output_path, f'{focals[idx]}.jpg'), stack[idx])

### Ablation comparison

In [ ]:
alternative_increasing_model = torch.load('PLACEHOLDER')
alternative_increasing_model = alternative_increasing_model.cuda(
) if torch.cuda.is_available() else alternative_increasing_model
alternative_increasing_model.eval()

focals = ['F-75', 'F-60', 'F-45', 'F-30', 'F-15',
            'F0', 'F15', 'F30', 'F45', 'F60', 'F75']

# Find all the stacks of interest
stack_dirs = glob.glob('PLACEHOLDER')[20:]
os.mkdir('ssim-comp')
# Loop through and generate
for i, stack_dir in enumerate(random.sample(stack_dirs, 30)):
    # Load the stack
    stack = preprocess_stack(load_stack(stack_dir))
    idx = random.randint(2, len(stack) - 1)
    plane = predict_with_model(
            increasing_model, stack[idx - 1], stack[idx - 2]
        ) * circle_mask
    plane_alt = predict_with_model(
            alternative_increasing_model, stack[idx - 1], stack[idx - 2]
        ) * circle_mask
    plane[1:20, 1:20] = 1
    plane_alt[1:20, 1:20] = 1
    # Save the new stack
    output_path = os.path.join('ssim-comp', str(i))
    os.mkdir(output_path)
    io.imsave(os.path.join(output_path, f'F{random.randint(100, 10000)}.jpg'), plane.numpy())
    io.imsave(os.path.join(output_path, f'F{random.randint(100, 10000)}.jpeg'), plane_alt.numpy())
    io.imsave(os.path.join(output_path, f'{focals[idx]}.jpg'), stack[idx])

In [ ]:
def predict_middle_focal(upper, lower):
    image = np.zeros((400, 400, 2))
    image[:, :, 0] = upper
    image[:, :, 1] = lower
    image = torch.from_numpy(image).permute(2, 0, 1).unsqueeze(0).float()
    image = image.cuda() if torch.cuda.is_available() else image
    pred = middle_model.forward(image)
    pred = pred.squeeze().detach().cpu() * circle_mask
    return pred


def generate_interpolated_stack(stack):
    new_stack = []
    for i in range(len(stack) - 1):
        new_stack.append(stack[i])
        new_stack.append(predict_middle_focal(stack[i], stack[i + 1]))
    new_stack.append(stack[len(stack) - 1])
    return new_stack

stack_dirs = glob.glob('PLACEHOLDER')[20:]
os.mkdir('superres')

focals = ['F-75', 'F-67', 'F-60', 'F-53', 'F-45', 'F-37', 'F-30', 'F-23', 'F-15', 'F-7',
            'F0', 'F7', 'F15', 'F23', 'F30', 'F37', 'F45', 'F53', 'F60', 'F67', 'F75']

# Loop through and generate
for i, stack_dir in enumerate(random.sample(stack_dirs, 10)):
    output_path = f'superres/{i}'
    os.mkdir(output_path)
    # Load the stack
    stack = preprocess_stack(load_stack(stack_dir))
    # Generate
    stack = generate_interpolated_stack(stack)
    # Save
    for j, plane in enumerate(stack):
        io.imsave(os.path.join(output_path, f'{focals[j]}.jpg'), plane)
    # fig, ax = plt.subplots(3, int(np.ceil(len(stack) / 3)))
    # fig.set_size_inches(21, 9)
    # for i in range(len(stack)):
    #     ax[int(np.floor(i / 7)), i % 7].imshow(stack[i])
    # fig.savefig('test.jpg', dpi=300)

### Segmentation Experiments

In [ ]:
# Find all the stacks of interest
stack_dirs = glob.glob('PLACEHOLDER')
os.mkdir('PLACEHOLDER')
os.mkdir('PLACEHOLDER')

with_segs = [
    # TODO: ADD PATHS
]

stack_dirs = [x for x in stack_dirs if os.path.basename(x) in with_segs]

# Loop through and generate
for i, stack_dir in enumerate(stack_dirs):
    # Load the stack
    stack = preprocess_stack(load_stack(stack_dir))
    plane_plus_90 = predict_with_model(
            increasing_model, stack[10], stack[9]
        ) * circle_mask
    plane_plus_105 = predict_with_model(
            increasing_model, plane_plus_90, stack[10]
        ) * circle_mask
    plane_plus_90[1:20, 1:20] = 1
    plane_plus_105[1:20, 1:20] = 1
    # Save the new stack
    output_path = os.path.join('PLACEHOLDER', os.path.basename(stack_dir))
    output_path_orig = os.path.join('PLACEHOLDER', os.path.basename(stack_dir))
    os.mkdir(output_path)
    os.mkdir(output_path_orig)
    io.imsave(os.path.join(output_path, 'F90.jpg'), plane_plus_90.numpy())
    io.imsave(os.path.join(output_path, 'F105.jpg'), plane_plus_105.numpy())
    # Add existing planes
    focals = ['F-45', 'F-30', 'F-15',
        'F0', 'F15', 'F30', 'F45', 'F60', 'F75']
    for j, img in enumerate(stack[2:]):
        io.imsave(os.path.join(output_path, f'{focals[j]}.jpg'), img)
    # Add existing planes
    focals = ['F-75', 'F-60', 'F-45', 'F-30', 'F-15',
        'F0', 'F15', 'F30', 'F45', 'F60', 'F75']
    for j, img in enumerate(stack):
        io.imsave(os.path.join(output_path_orig, f'{focals[j]}.jpg'), img)